# Using TZIP-16 and off-chain views with PyTezos

In this notebook, we're going to improve the first example of contract that we wrote (TODO: first chapter is not written yet ☺) and add TZIP-16 style metadata to the storage. We're going to use these metadata to store an _off-chain view_, which is a short piece of Michelson code that can be called to inspect the contract's storage.

In [1]:
import pytezos as tz
from pytezos.contract.interface import ContractInterface
from pytezos.contract.result import OperationResult

Let's first define a PyTezos client. As usual, you can change the following RPC to point to e.g., `https://ghostnet.tezos.marigold.dev` to use Ghostnet instead.

In [2]:
TEZOS_RPC="http://localhost:20000"

alice = tz.Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")
ptz = tz.pytezos.using(TEZOS_RPC, alice)

In [3]:
ptz.balance()

Decimal('1999846.298958')

## A simple example of contract

As in the first chapter, we define a simple contract that stores, in a big map, one integer counter per address. These counters can be incremented or decremented by calling the corresponding entrypoints.

We save this code to a file called `test.mligo`.

In [4]:
%%writefile test.mligo

type storage = {
  counters: (address, int) big_map;
  metadata: (string, bytes) big_map;
}

type action =
  | Increment of address
  | Decrement of address

let update_with (i: int) (address: address) (storage: storage) =
  match Big_map.find_opt address storage.counters with
    | None -> 
      { storage with counters = Big_map.add address i storage.counters }
    | Some j -> 
      { storage with counters = Big_map.add address (i+j) storage.counters }

let main (action, storage): operation list * storage =
  match action with
    | Increment address ->
      let updated_storage = update_with 1 address storage in
      ([], updated_storage)
    | Decrement address ->
      let updated_storage = update_with (-1) address storage in
      ([], updated_storage)


Overwriting test.mligo


We added a `metadata` field to the storage, which is a big_map with `string` keys and `bytes` values. Bytes are used to circumvent the limitations of Michelson strings.

If we call the LIGO compiler, we get the following Michelson code:

In [5]:
! ligo compile contract test.mligo

{ parameter (or (address %decrement) (address %increment)) ;
  storage (pair (big_map %counters address int) (big_map %metadata string bytes)) ;
  code { LAMBDA
           (pair int address (big_map address int) (big_map string bytes))
           (pair (big_map address int) (big_map string bytes))
           { UNPAIR 3 ;
             DUP 3 ;
             CAR ;
             DUP 3 ;
             GET ;
             IF_NONE
               { DUP 3 ;
                 DIG 3 ;
                 CAR ;
                 DIG 2 ;
                 DIG 3 ;
                 SWAP ;
                 SOME ;
                 SWAP ;
                 UPDATE ;
                 UPDATE 1 }
               { DUP 4 ;
                 DIG 4 ;
                 CAR ;
                 DIG 2 ;
                 DIG 3 ;
                 ADD ;
                 DIG 3 ;
                 SWAP ;
                 SOME ;
                 SWAP ;
                 UPDATE ;
                 UPDATE 1 } } ;
         SWAP ;
         U

## Writing an off-chain view

The previous contract allocates storage (on-chain) for the `counters` big map, where it has all the information it needs to operate. However, big maps are a bit hard to inspect, as they don't store the keys to the values but only a hash thereof. Inspecting the contract storage without an indexer (such as TzKt API) is thus very hard.

_Views_ are short functions that can be added to a smart contract to access some parts of the storage, or to check some conditions based on a few computations. They can be called by other contracts _on-chain_ or through Tezos RPC nodes, depending on what view the RPC node has of the contract.

Let's imagine that you're writing a trading application on-chain, such as a DEX. The price of a token at a given block is defined by on-chain information (such as buy and sell orders for an orderbook, or the state of the liquidity pool for a CFMM DEX). However, if you want to put a nice page for analytics that shows the prices on your DEX, you need to inspect the state of the contract through the RPC, compute the price and show it on the page. Views typically allow to write this kind of code, that focus on a part of the contract's storage only.

In our case, we can write a view that checks what the value of a counter is, for a given address. The view only reads from the big map, it doesn't change it. Let's append the following view to our file.

In [6]:
%%writefile -a test.mligo

let get_storage ((address, storage): address * storage): int option =
  Big_map.find_opt address storage.counters

Appending to test.mligo


If we compile the contract again, we can see that the compiler produces the same code as before: the `get_storage` view hasn't been added to the contract at all!

In [7]:
! ligo compile contract test.mligo

{ parameter (or (address %decrement) (address %increment)) ;
  storage (pair (big_map %counters address int) (big_map %metadata string bytes)) ;
  code { LAMBDA
           (pair int address (big_map address int) (big_map string bytes))
           (pair (big_map address int) (big_map string bytes))
           { UNPAIR 3 ;
             DUP 3 ;
             CAR ;
             DUP 3 ;
             GET ;
             IF_NONE
               { DUP 3 ;
                 DIG 3 ;
                 CAR ;
                 DIG 2 ;
                 DIG 3 ;
                 SWAP ;
                 SOME ;
                 SWAP ;
                 UPDATE ;
                 UPDATE 1 }
               { DUP 4 ;
                 DIG 4 ;
                 CAR ;
                 DIG 2 ;
                 DIG 3 ;
                 ADD ;
                 DIG 3 ;
                 SWAP ;
                 SOME ;
                 SWAP ;
                 UPDATE ;
                 UPDATE 1 } } ;
         SWAP ;
         U

As it does not change the code produced, there's no need to add it to the same file. However, doing so is still often a good idea for two reasons:
* the code of the view depends on the storage, and even expresses some basic property about it (e.g., how price are computed or what a counter is). It explains the rest of the code as well;
* it can still be used in tests, through a library such as [Breathalyzer](https://github.com/marigold-dev/breathalyzer).

Still, this view is not part of the contract's code and is not required to be deployed on-chain. It cannot be called through a transaction, only through a simulation, ran by a RPC node. For these reasons, this type of views are called _off-chain_.

## Compiling and storing the view

Off-chain views are defined by the [TZIP-16](https://tzip.tezosagora.org/proposal/tzip-16/), which describes the metadata of a contract. Such metadata is stored as a JSON object with a few fields (some mandatory, some optional), which can be accessed through a link, itself stored in a big map inside the contract's storage.

However, this link can itself point to the same big map, by using the `tezos-storage:` URL prefix!

In the following, we're going to show how to compile the view as a JSON expression and deploy it with the contract.

In [8]:
view_code = !ligo compile expression cameligo get_storage --init-file main.mligo --michelson-format json

In [9]:
import json
view_code = json.loads(view_code.spstr)   # view_code.spstr is just getting the output of the compiler

In [10]:
view_code

[{'prim': 'UNPAIR'},
 {'prim': 'SWAP'},
 {'prim': 'CAR'},
 {'prim': 'SWAP'},
 {'prim': 'GET'}]

Two things to quickly note here:
* as the code of the view is really short, so is the Michelson code produced by the compiler;
* the code is in [JSON-seralized Micheline](https://tezos.gitlab.io/shell/micheline.html) format instead of the usual Michelson.

Views serialization follows a specific format as well. Writing the types can sometimes be tricky, but is required to make sure that the RPC nodes and libraries such as PyTezos can correctly communicate.

In [11]:
views = [
    {
        "name": "get_storage",
        "parameter": {
            "prim": "address"
        },
        "returnType": {
            "prim": "option", 
            "args": [ { "prim": "int" } ]
        },
        "code": view_code
    }
]

Finally, the full metadata JSON can be defined as such. Follow the link towards TZIP-16 for more information about the available implementations for views.

In [12]:
tzip16_metadata =  {
    "interfaces": [],
    "views": [
            {
                "name": view["name"],
                "implementations": [
                    {
                        "michelsonStorageView": {
                            "parameter": view["parameter"],
                            "returnType": view["returnType"],
                            "code": view["code"],
                        }
                    }
                ],
            }
            for view in views
        ]
    }

As metadata are stored as bytes, we have to encode them. The `metadata` big map that we are going to store has two entries: the first one simply points towards the second one (as a URI), but other URIs would have been allowed, such as an IPFS or even simple HTTP links.

In [13]:
metadata = json.dumps(tzip16_metadata).encode("utf-8")
storage_metadata = {
    "": b"tezos-storage:m".hex(),
    "m": metadata,
}

We can finally deploy our contract with the initial values for the two big maps:

In [14]:
contract = ContractInterface.from_file("./main.tz").script(
    initial_storage=[{}, storage_metadata]
)

In [15]:
blocks = ptz.shell.blocks()
opg = ptz.origination(contract, balance=0).fill(ttl=1).sign().send()
opg = ptz.wait(opg, min_confirmations=1, prev_hash=ptz.shell.blocks[-2]()["hash"])

In [16]:
opg_result = OperationResult.from_operation_group(opg[0])[0]

In [17]:
# To build a ContractInterface object from the freshly deployed one, we need its address
contract = ptz.contract(opg_result.originated_contracts[0])

Note that PyTezos correctly parses the stored metadata. However, if we access the big map directly, we get bytes.

In [18]:
contract.metadata()

{'interfaces': [],
 'views': [{'name': 'get_storage',
   'implementations': [{'michelsonStorageView': {'parameter': {'prim': 'address'},
      'returnType': {'prim': 'option', 'args': [{'prim': 'int'}]},
      'code': [{'prim': 'UNPAIR'},
       {'prim': 'SWAP'},
       {'prim': 'CAR'},
       {'prim': 'SWAP'},
       {'prim': 'GET'}]}}]}]}

In [19]:
metadata_big_map = contract.storage["metadata"]()
ptz.shell.head.context.big_maps[metadata_big_map]()

[{'bytes': '74657a6f732d73746f726167653a6d'},
 {'bytes': '7b22696e7465726661636573223a205b5d2c20227669657773223a205b7b226e616d65223a20226765745f73746f72616765222c2022696d706c656d656e746174696f6e73223a205b7b226d696368656c736f6e53746f7261676556696577223a207b22706172616d65746572223a207b227072696d223a202261646472657373227d2c202272657475726e54797065223a207b227072696d223a20226f7074696f6e222c202261726773223a205b7b227072696d223a2022696e74227d5d7d2c2022636f6465223a205b7b227072696d223a2022554e50414952227d2c207b227072696d223a202253574150227d2c207b227072696d223a2022434152227d2c207b227072696d223a202253574150227d2c207b227072696d223a2022474554227d5d7d7d5d7d5d7d'}]

Finally, you may have noticed that ContractInterface objects have a `views` property. It corresponds to on-chain views, which we explore in a different notebook.

In [20]:
contract.views

{}

## Calling an off-chain view

Off-chain views are rather poorly documented in the Tezos ecosystem, and in PyTezos documentation in particular. Fortunately, the introspective nature of Python and PyTezos still make it somewhat easy to discover these views and call them.

While PyTezos parses the `contract.metadata` property for us, this is not how you should access the views:

In [21]:
offchain_views = contract.metadata()["views"]

In [22]:
offchain_views

[{'name': 'get_storage',
  'implementations': [{'michelsonStorageView': {'parameter': {'prim': 'address'},
     'returnType': {'prim': 'option', 'args': [{'prim': 'int'}]},
     'code': [{'prim': 'UNPAIR'},
      {'prim': 'SWAP'},
      {'prim': 'CAR'},
      {'prim': 'SWAP'},
      {'prim': 'GET'}]}}]}]

Instead, let's use the fact that PyTezos produces a ContractMetadata object with all the fields that we need:

In [23]:
contract.metadata


Properties
.key		tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb
.shell		['http://localhost:20000']
.address	KT1CUSGvAHXFZzCfNLdCfT1Rpy4uJG9kzjC6
.block_id	head

Metadata
.name		
.description	
.license	
.authors	
.interfaces	
.errors		<0>
.views		<1>

Storage views
.getStorage()

As you can see, we can either explore the types and code of the views like this…

In [24]:
v = contract.metadata.views[0]
m = v.implementations[0]
m.michelsonStorageView

MichelsonStorageView(parameter={'prim': 'address'}, returnType={'prim': 'option', 'args': [{'prim': 'int'}]}, code=[{'prim': 'UNPAIR'}, {'prim': 'SWAP'}, {'prim': 'CAR'}, {'prim': 'SWAP'}, {'prim': 'GET'}], annotations=None, version=None)

…or directly call a “storage view” with the relevant arguments, which correctly uses the updated contract storage.

In [25]:
contract.metadata.getStorage(alice.public_key_hash()).storage_view()

As we haven't interacted with our freshly deployed contract yet, the `counters` big map is still empty and we get a `None`. Let's call the entrypoints a bit!

In [27]:
tx1 = contract.increment(alice.public_key_hash()).as_transaction().autofill().sign().inject(min_confirmations=1)
tx2 = contract.increment(alice.public_key_hash()).as_transaction().autofill().sign().inject(min_confirmations=1)

In [29]:
contract.metadata.getStorage(alice.public_key_hash()).storage_view()

2